In [151]:
import os
from dotenv import load_dotenv
import json
import pandas as pd
import traceback


In [152]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

In [153]:
load_dotenv()
api_key=os.getenv("HUGGINGFACEHUB_API_TOKEN")
os.environ["HUGGINGFACEHUB_API_TOKEN"]=api_key

In [170]:
llm=HuggingFaceHub(repo_id="meta-llama/Llama-2-7b-chat-hf",model_kwargs={"temperature": 0.7, "max_length": 512})

In [171]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

import PyPDF2

In [172]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above test, it is your job to \
Create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be confirming the test as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [173]:
RESPONSE_JSON={
    "1":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "2":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "3":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
}

In [174]:
prompt=PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template=TEMPLATE
)

In [175]:
chain=LLMChain(llm=llm,prompt=prompt,output_key="quiz",verbose=True)

In [176]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [177]:
prompt2=PromptTemplate(
    input_variables=['subject','quiz'],
    template=TEMPLATE2
)

In [178]:
chain2=LLMChain(llm=llm,prompt=prompt2,output_key="review",verbose=True)

In [179]:
generate_chain=SequentialChain(chains=[chain,chain2],input_variables=['text','number','subject','tone','response_json'],output_variables=['quiz','review'],verbose=True)

In [180]:
file_path="C:\\Users\\SHREYA SINGH\\OneDrive\\Desktop\\MCQ_Generator\\data.txt"

In [181]:
with open(file_path,'r') as file:
    TEXT=file.read()

In [182]:
TEXT

'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.\n\nStatistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.\n\nFrom a theoretical viewpoint, probably approximately correc

In [183]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [184]:
NUMBER=5
SUBJECT="machine learning"
TONE='simple'

In [185]:
response=generate_chain(
    {
        "text":TEXT,
        "number":NUMBER,
        "subject":SUBJECT,
        "tone":TONE,
        "response_json":json.dumps(RESPONSE_JSON)
    }
)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data a

c:\Users\SHREYA SINGH\OneDrive\Desktop\MCQ_Generator\env\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for machine learning students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approa

c:\Users\SHREYA SINGH\OneDrive\Desktop\MCQ_Generator\env\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



> Finished chain.

> Finished chain.


In [190]:
import re

match = re.search(r"### YOUR RESPONSE\s*(\{.*\})", response.get('quiz'), re.DOTALL)
quiz=json.loads(str(match.group(1)))

In [192]:
quiz

{'1': {'mcq': 'What is machine learning?',
  'options': {'a': 'A type of data analysis that draws conclusions from data without being explicitly programmed.',
   'b': 'A type of programming that involves writing code to solve complex problems.',
   'c': 'A type of software development that focuses on creating user-friendly interfaces.',
   'd': 'A type of data visualization that helps in understanding complex data.'},
  'correct': 'a'},
 '2': {'mcq': 'Which of the following is NOT a part of machine learning?',
  'options': {'a': 'Supervised Learning',
   'b': 'Unsupervised Learning',
   'c': 'Reinforcement Learning',
   'd': 'Programmed Learning'},
  'correct': 'd'},
 '3': {'mcq': 'What does ML find application in?',
  'options': {'a': 'Only in programming and data analysis.',
   'b': 'In many fields like natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.',
   'c': 'Only in business problems as predictive analytics.',
   'd': '

In [193]:
quiz_table_data=[]
for key, value in quiz.items():
    mcq=value['mcq']
    options=" | ".join(
        [
            f"{option}:{option_value}"
            for option, option_value in value['options'].items()
        ]
    )
    correct=value['correct']
    quiz_table_data.append({"MCQ":mcq,"Choices":options,"Correct":correct})

In [194]:
quiz_table_data

[{'MCQ': 'What is machine learning?',
  'Choices': 'a:A type of data analysis that draws conclusions from data without being explicitly programmed. | b:A type of programming that involves writing code to solve complex problems. | c:A type of software development that focuses on creating user-friendly interfaces. | d:A type of data visualization that helps in understanding complex data.',
  'Correct': 'a'},
 {'MCQ': 'Which of the following is NOT a part of machine learning?',
  'Choices': 'a:Supervised Learning | b:Unsupervised Learning | c:Reinforcement Learning | d:Programmed Learning',
  'Correct': 'd'},
 {'MCQ': 'What does ML find application in?',
  'Choices': 'a:Only in programming and data analysis. | b:In many fields like natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. | c:Only in business problems as predictive analytics. | d:Only in statistics and mathematical optimization.',
  'Correct': 'b'},
 {'MCQ': "Who coined 

In [195]:
quiz_df=pd.DataFrame(quiz_table_data)

In [196]:
quiz_df

,MCQ,Choices,Correct
0,What is machine learning?,a:A type of data analysis that draws conclusio...,a
1,Which of the following is NOT a part of machin...,a:Supervised Learning | b:Unsupervised Learnin...,d
2,What does ML find application in?,a:Only in programming and data analysis. | b:I...,b
3,Who coined the term 'machine learning'?,a:Donald Hebb | b:Arthur Samuel | c:Warren McC...,b
4,What is a key mathematical foundation of machi...,a:Algebra | b:Geometry | c:Statistics and math...,c


In [199]:
quiz_df.to_csv("Machine_learning.csv",index=False)